# Pseudocode zur Implementierung

In [27]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [41]:
# dieser Test-df wird von Christophs Programm erzeugt.
# wichtig!!! Bei der Besprechung müssen wir P_delta als NaN Value in den Datenframe einfügen
P_Phase = pd.read_csv('power_today_minute_1.csv', names = ['timestamp','P', 'P_delta'])

#Event dataframe anlegen zur späteren Auswertung der An/Abschaltevents
Event_df = pd.DataFrame(columns = ['timestamp','Phase','minuten_index',
                                   'Ladevorgang', 'Status', 'Ladeleistung',
                                   'minuten_index_Abschaltung','timestamp_abschalt'])

## Funktion
-------

In [42]:
def add_p_delta(df):
    df_length = len(df)
    index_last_row = df_length - 1
    if(df_length > 1):
        p_new = df.iloc[index_last_row]['P']
        p_old = df.iloc[index_last_row-1]['P']
        p_delta = p_new - p_old
        return p_delta
    else:
        return 0

In [43]:
def detect_switch_event(rows, phase, Event_df):
    minuten_index = rows.index[-1]
    ### Einschaltvorgang dedektieren ###
    # ist delta P relevant
    if(rows.iloc[-1]['P_delta'] > 400):
        timestamp = rows.iloc[0]['timestamp']
        # Prüfen ob Ladenvorgang beginnt/beendet
        # Wenn delta P positiv ---> event kreieren um später zu checken ob es ein Einschaltvorgang war
        Event_df.loc[len(Event_df)]= [timestamp, phase, minuten_index, 'ein', 'not checked', 0,0,0]
    
    #Check events: Überprüfen ob vorher eingetragenes event wirklich ein Einschaltvorgang ist
    event_df_length = len(Event_df)
    row_event = 0
    while row_event < event_df_length:
            #condition to check event
            minuten_index_event = Event_df.iloc[row_event]['minuten_index']
            #Check auf Einschaltvorgang
            if ((minuten_index_event+3 == minuten_index) & 
                (Event_df.iloc[row_event]['Ladevorgang'] == 'ein') &
                (Event_df.iloc[row_event]['Phase'] == phase)):
                #display(Event_df)
                #display(rows)
                Ladeleistung = rows.loc[minuten_index_event+3]['P'] - rows.loc[minuten_index_event-3]['P']            
                sum_delta_P = rows.loc[minuten_index_event:minuten_index_event+3]['P_delta'].sum()
                deviation = abs(Ladeleistung-sum_delta_P)/Ladeleistung 
                #print('Minute' + str(row))
                #print(Event_df)
                print('Ladeleistung: ' + str(Ladeleistung))
                print('sum_delta_P: ' + str(sum_delta_P))
                print('deviation' + str(deviation))
                display(Event_df)
                if ((deviation > 0.15) | (Ladeleistung < 1500)):
                    Event_df = Event_df.drop(row_event)
                    Event_df = Event_df.reset_index(drop=True)
                else:
                    Event_df.loc[row_event, ['Status', 'Ladeleistung']] = ['checked', Ladeleistung]
            event_df_length = len(Event_df)
            row_event += 1
    '''     
    ### Ausschaltvorgang dedektieren ###
    #Checken ob aktuelle Leistung das Ende eines Einschaltvorgangs impliziert        
    if (('ein' in Event_df['Ladevorgang'].unique()) &
        ('checked' in Event_df['Status'].unique())):
        # create df with charging status on
        only_load_event_df = Event_df[(Event_df['Ladevorgang'] == 'ein') &
                                      (Event_df['minuten_index'] < minuten_index)]
        # Soll Leistung = Leistung die auf der Phase anliegen sollte,
        # wenn alle zuvor ladenden Autos noch an der Phase hängen würden
        Soll_Leistung = Leerlaufleistung_Phase + Event_df[(Event_df['Ladevorgang']=='ein') & 
                                                          (Event_df['minuten_index'] <= minuten_index) & 
                                                          (Event_df['minuten_index_Abschaltung'] == 0) & 
                                                          (Event_df['Phase'] == Phase)]['Ladeleistung'].sum()
        #Aktuelle Leistung (neuer Messwert)
        Ist_Leistung = P_new 
        #Residuale Leistung 
        P_residual = Soll_Leistung - Ist_Leistung
        #Überprüfen ob fehlende Leistung (P_residual) zu einem zuvor ladenden Auto passt
        only_load_event_df['deviation_P'] = only_load_event_df['Ladeleistung']. \
                                    apply(lambda ladeleistung_auto: abs(ladeleistung_auto-P_residual)/ladeleistung_auto)
        if ((only_load_event_df['deviation_P'] < 0.03).any()):
            # sort load events by size
            # und nehme das erste Event, was der aktuellen leistung am nächsten ist
            charg_off_ev_index = only_load_event_df['deviation_P'].sort_values().index[0]
            Event_df.loc[charg_off_ev_index,['minuten_index_Abschaltung']] = minuten_index
            Event_df.loc[charg_off_ev_index,['timestamp_abschalt']] = Timestamp
            Event_df.loc[charg_off_ev_index, ['Ladevorgang']] = 'aus'
    '''    
    return Event_df

## Code Simulation auf dem Server
----
Hier kann der Code für eine Phase Simuliert werden

In [44]:
pseudocode_df = pd.DataFrame([])
phasen_variable = 1

for row in range(0, 900):
    pseudocode_df = pseudocode_df.append(P_Phase[row:row+1])
    #print('----------Zeile {} ----------'.format(row))
    #print('Eingehender Dataframe:')
    #display(pseudocode_df)
    
    # call the function to add P-delta
    pseudocode_df.iloc[-1, pseudocode_df.columns.get_loc('P_delta')] = add_p_delta(pseudocode_df[-2:])
    # Detect all switch events
    Event_df = detect_switch_event(pseudocode_df.iloc[-7:], 2, Event_df)
    '''
    Hier wird soll die Funktion Ein-/ Ausschaltvorgang implementiert werden,
    die unseren Event_df erzeugt.
    '''
   
    #print('Dataframe mit P_delta:')
    #display(pseudocode_df)
    #print()
    #print()

Ladeleistung: 3590.0
sum_delta_P: 3584.0
deviation0.0016713091922


,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505390e+09,2,834,ein,not checked,0,0,0
1,1.505390e+09,2,835,ein,not checked,0,0,0


Ladeleistung: 3587.0
sum_delta_P: 1819.0
deviation0.492890995261


,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505390e+09,2,834,ein,checked,3590,0,0
1,1.505390e+09,2,835,ein,not checked,0,0,0


Ladeleistung: 3630.0
sum_delta_P: 3626.0
deviation0.00110192837466


...

,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505390e+09,2,834,ein,checked,3590,0,0
1,1.505391e+09,2,860,ein,checked,3630,0,0
2,1.505391e+09,2,861,ein,not checked,0,0,0


Ladeleistung: 3438.0
sum_delta_P: 3426.0
deviation0.00349040139616


,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505390e+09,2,834,ein,checked,3590,0,0
1,1.505391e+09,2,860,ein,checked,3630,0,0
2,1.505393e+09,2,888,ein,not checked,0,0,0
3,1.505393e+09,2,889,ein,not checked,0,0,0


Ladeleistung: 3419.0
sum_delta_P: 2474.0
deviation0.276396607195


,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505390e+09,2,834,ein,checked,3590,0,0
1,1.505391e+09,2,860,ein,checked,3630,0,0
2,1.505393e+09,2,888,ein,checked,3438,0,0
3,1.505393e+09,2,889,ein,not checked,0,0,0


In [45]:
Event_df

,timestamp,Phase,minuten_index,Ladevorgang,Status,Ladeleistung,minuten_index_Abschaltung,timestamp_abschalt
0,1.505390e+09,2,834,ein,checked,3590,0,0
1,1.505391e+09,2,860,ein,checked,3630,0,0
2,1.505393e+09,2,888,ein,checked,3438,0,0


In [ ]:
pseudocode_df[pseudocode_df['P_delta'] > 200]